# In Class Lab 1

### Due by midnight, thursday in your github repository 'Labs/Lab1' folder


In [10]:
# Import Modules 
import numpy as np # import numpy
import astropy.units as u # import astropy units
from astropy import constants as const # import astropy constants

### Astropy Units:  https://docs.astropy.org/en/stable/units/index.html


## Part A:  The Local Standard of Rest
Proper motion of Sgr A* from Reid & Brunthaler 2004
$\mu = 6.379$ mas/yr 

Peculiar motion of the sun, $v_\odot$ = 12.24 km/s  (Schonrich 2010)


$v_{tan} = 4.74 \frac{\mu}{\rm mas/yr} \frac{R_o}{\rm kpc} = V_{LSR} + v_\odot$


### a)

Create a function called VLSR to compute the local standard of res (V$_{LSR}$).

The function should take as input: the solar radius (R$_o$), the proper motion (mu)
and the peculiar motion of the sun in the $v_\odot$ direction.

Compute V$_{LSR}$ using three different values R$_o$: 
1. Water Maser Distance for the Sun :  R$_o$ = 8.34 kpc   (Reid 2014 ApJ 783) 
2. GRAVITY Collaboration Distance for the Sun:  R$_o$ = 8.178 kpc   (Abuter+2019 A&A 625)
3. Value for Distance to Sun listed in Sparke & Gallagher : R$_o$ = 7.9 kpc 


In [11]:
#4.74*mu*Ro = VLSR + vsun
def VLSR(Ro, mu=6.379, vsun=12.24*u.km/u.s):
    """This function will compute the velocity at the local standard of rest
        VLSR = 4.74*mu+Ro - vsun
        Imputs: Ro (astropy units kpc) Distance from the sun to the galactic center
                mu is the proper motion of Sag A star (mas/year)
                vsun (astropy units km/s) the peculiar motion of tyhe sun in the v direction
        Outputs: VLSR (astropy units km/s) The local standard rest
    """
    vlsr = 4.74*mu*(Ro/u.kpc)*u.km/u.s - vsun
    return vlsr
    


In [12]:
RoReid = 8.34*u.kpc #Reid + 2014
RoAbuter = 8.178*u.kpc #Gravity Abuter+2019
RoSparke = 7.9*u.kpc #Spark and Gallagher Text

In [13]:
#Compute VLSR using Reid 2014
VLSR_Reid = VLSR(RoReid)
print(VLSR_Reid)

239.9320764 km / s


In [16]:
#Compute VLSR from Sparke and Gallagher
VLSR_Sparke = VLSR(RoSparke)
print(VLSR_Sparke)

226.628034 km / s


In [17]:
#Compute VLSR using gravity collaborator

VLSR_Abuter = VLSR(RoAbuter)
print(VLSR_Abuter)
print(np.round(VLSR_Abuter))

235.03376988000002 km / s
235.0 km / s


### b)

compute the orbital period of the sun in Gyr using R$_o$ from the GRAVITY Collaboration (assume circular orbit)

Note that 1 km/s $\sim$ 1kpc/Gyr

In [27]:
#Orbital period = 2piR/V

def TorbSun(Ro, Vc):
    """ Function that computes the Orbital Period
    T = 2pi R / V
    Inputes: Ro (astropy quantity) distance to the galactic center from the sun (kpc)
            Vc (astropy quantity) velocity of the sun in the "v" direction (km / s)
    Outputs: T(astropy quantity) Orbital Period (Gyr)
    """
    VkpcGyr = Vc.to(u.kpc/u.Gyr) #Converting V to kpc to Gyr
    T= 2*np.pi*Ro/VkpcGyr #Orbital period
    return T

In [28]:
VsunPec = 12.24*u.km/u.s #peculiar motion

In [29]:
Vsun = VLSR_Abuter + VsunPec #Total motion of the sun in "v" direction

In [33]:
#Orbital period of the sun
T_Abuter = TorbSun(RoAbuter, Vsun)
print(T_Abuter)
print(np.round(T_Abuter,3))

0.20318680562272234 Gyr
0.203 Gyr


### c)

Compute the number of rotations about the GC over the age of the universe (13.8 Gyr)

In [36]:
AgeUniverse = 13.8*u.Gyr
print(AgeUniverse/T_Abuter)

67.91779593023313


## Part B  Dark Matter Density Profiles

### a)
Try out Fitting Rotation Curves 
[here](http://wittman.physics.ucdavis.edu/Animations/RotationCurve/GalacticRotation.html)


### b)


In the Isothermal Sphere model, what is the mass enclosed within the solar radius (R$_o$) in units of M$_\odot$? 

Recall that for the Isothermal sphere :
$\rho(r) = \frac{V_{LSR}^2}{4\pi G r^2}$

Where $G$ = 4.4985e-6 kpc$^3$/Gyr$^2$/M$_\odot$, r is in kpc and $V_{LSR}$ is in km/s

What about at 260 kpc (in units of  M$_\odot$) ? 

In [38]:
print(const.G)

  Name   = Gravitational constant
  Value  = 6.6743e-11
  Uncertainty  = 1.5e-15
  Unit  = m3 / (kg s2)
  Reference = CODATA 2018


In [40]:
Grav = const.G.to(u.kpc**3/u.Gyr**2/u.Msun)
print(Grav)

4.498502151469554e-06 kpc3 / (solMass Gyr2)


In [42]:
#Density profile rho = VLSR^2/(4pi *G *R^2)
#Mass (r) = Integrate rho dV
#          Integrate rho 4pi*r^2*dr
#           Integrate VLSR^2 / (pi*G*r^2)*4pi*r^2 dr
# .          VLSR^2/G *r

def massIso(r, VLSR):
    ''' This function will compute the dark matter mass enclosed
    within a given distance r, assuming an isothermal sphere model
    M(r) = VLSR^2/G *r

    Inputs:
        r (astropy quantity) distance from galactic center (kpc)
        VLSR (astropy quantity) the velocity at the local standard of rest
        in (km/s)
    Outputs:
        M (astropy quantity) mass enclosed within r (Msun)
    '''

    VLSRkpcGyr = VLSR.to(u.kpc/u.Gyr) #translates to kpc/Gyr
    M = VLSRkpcGyr**2/Grav*r #Isothermal sphere mass profile
    return M

In [44]:
#compute the mass enclosed within Ro (Gravity Collab)
mIsoSolar = massIso(RoAbuter, VLSR_Abuter)
print(mIsoSolar)
print(f"{mIsoSolar:.2e}")

105038025820.79904 solMass
1.05e+11 solMass


In [47]:
#compute the mass enclosed at 260 kpc
mIso260 = massIso(260*u.kpc, VLSR_Abuter)
print(f"{mIso260:.2e}")

3.34e+12 solMass


## c) 

The Leo I satellite is one of the fastest moving satellite galaxies we know. 


It is moving with 3D velocity of magnitude: Vtot = 196 km/s at a distance of 260 kpc (Sohn 2013 ApJ 768)

If we assume that Leo I is moving at the escape speed:

$v_{esc}^2 = 2|\Phi| = 2 \int G \frac{\rho(r)}{r}dV $ 

and assuming the Milky Way is well modeled by a Hernquist Sphere with a scale radius of $a$= 30 kpc, what is the minimum mass of the Milky Way (in units of M$_\odot$) ?  

How does this compare to estimates of the mass assuming the Isothermal Sphere model at 260 kpc (from your answer above)

In [57]:
#potential for a Hernquist shpere
#. Phi = -G*M/ (r+a)

# Escape speed becomes:
# vesc^2 = 2*G*M/(r+a)
#M = vesc**2/2/G*(r+a)
def massHernVesc(vesc, r, a=30*u.kpc):
    """
    This function determiens the total dark matter needed given an escape speed, assuming a Hernquist profile
        M = vesc^2/2/G*(r+a)

    Inputs:
        vesc (astropy quantity) escape speed (or speed of satellite) (km/s)
        r: (astropy quantity) distance from the galactic center (kpc)
        a: (astropy quantity) the hernquist scale length (kpc)
            default value of 30kpc
    Outputs:
        M (astropy quantity) mass within r (Msun)
    """
    vescKpcGyr = vesc.to(u.kpc/u.Gyr) #translates to kpc/Gyr
    M = vescKpcGyr**2/2/G*(r+a)
    return M

In [58]:
Vleo = 196*u.km/u.s #speed of leo I Sohn et al.
r = 260*u.kpc

In [59]:
MLeoI = massHernVesc(Vleo, r)
print(f"{MLeoI:.2e}")

NameError: name 'G' is not defined

In [ ]:
mIso260/MLeoI